# Model 

In [1]:
import io
import os
import pandas as pd
import  docx2txt
from docx import Document
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import string # special operations on strings
import spacy # language models
from matplotlib.pyplot import imread
from matplotlib import pyplot as plt
from wordcloud import WordCloud
%matplotlib inline
import win32com.client
import warnings
warnings.filterwarnings('ignore')
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.metrics import f1_score,precision_score,confusion_matrix,recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [2]:
import textract
import re #regular expression
import string
from sklearn.preprocessing import LabelEncoder

In [3]:
from nltk.corpus import stopwords
import pprint
from spacy.matcher import Matcher
import multiprocessing as mp
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer


In [4]:

def clean_text(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub("[0-9" "]+"," ",text)
    text = re.sub('[‘’“”…]', '', text)
    return text

clean = lambda x: clean_text(x)
def cleaned_review(review_df):
    review_df.replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r","[|]"], value=["","",""], regex=True, inplace=True)
    review_df=review_df.drop(index=[0,1,2,3,4,5,6,7,8,9],axis=0)
    review_df=review_df.reset_index(drop=bool)
    return review_df
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

def extract_skills(nlp_text):
    '''
    Helper function to extract skills from spacy nlp text
    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param noun_chunks: noun chunks extracted from nlp text
    :return: list of skills extracted
    '''
    tokens = nlp_text
    data = pd.read_csv('E:\DATA SETS\Resumes\Skill.csv')
    skills = list(data.columns.values)
    skillset = []
    # check for one-grams
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    return np.unique(skillset)
    # check for bi-grams and tri-grams
def extract_post(nlp_text):
    '''
    Helper function to extract skills from spacy nlp text
    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param noun_chunks: noun chunks extracted from nlp text
    :return: list of skills extracted
    '''
    tokens = nlp_text
    data = pd.read_csv('E:\DATA SETS\Resumes\post.csv')
    skills = list(data.columns.values)
    skillset = []
    # check for one-grams
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    return np.unique(skillset)
    # check for bi-grams and tri-grams
def lem(x):
    lmtzr = WordNetLemmatizer()
    lemmatized = [lmtzr.lemmatize(word) for word in x]
    return lemmatized  

def stem(x):
    ps = PorterStemmer()
    stemming = [ps.stem(word) for word in x]
    return stemming   

def extract_designation(x):
    y =''
    for token in x:
        if 'develop' in x and 'sql' in x and 'oracl' in x:
            y ='SQL and Oracle Developer'
        elif 'consult' in x and 'workday' in x:
            y= ' workday consultant'
        elif 'react' in x and 'develop' in x:
             y ='React  Developer'
        elif 'reactjs' in x and 'develop' in x:
             y ='ReactJS  Developer' 
        elif 'develop' in x and 'sql' in x :
            y ='SQL  Developer'
        elif 'hr' in x:
            y = 'human resource'
        elif 'peoplesoft'in x and 'admin' in x:
            y = 'PeopleSoft Admin'
        elif 'peoplesoft' in x and 'consult' in x:
            y ='PeopleSoft Consultant'
        elif token =='softwar':
             y ='Software Engineer'
        elif 'develop' in x and 'sql' in x :
            y ='SQL  Developer' 
        else:
            y ='intern'
    return y
def contactDetails(Text):
    name = ''  
    for i in range(0,3):
        name = " ".join([name, Text[i]])
    return(name)

In [5]:
def resume_classifier(source_directory):
    file_docx = []
    file_path = []
    for file_folder in os.listdir( source_directory):
    #final_path="E:/DATA SETS/Resumes/"+file_folder+".docx"
        final_path = os.path.join( source_directory, file_folder)
        if final_path.endswith('.docx') :
    #for j in os.listdir(os.path.join(r'E:/DATA SETS/Resumes', file_folder)):
        #final_folder_path = os.path.join(final_path,j)
            file_docx.append(textract.process(final_path))
            file_path.append(file_folder)
    df1 = pd.DataFrame(file_docx)
    df1['new_column'] = df1.apply(remove_punctuations)
    df1['Text'] = df1.new_column.str.decode("UTF-8") 
    cleanreviewdf1=cleaned_review(df1)
    df1.Text.drop_duplicates()
    cleaned_df = df1.Text.apply(clean)
    data = pd.concat([df1,cleaned_df],axis = 1)
    stop = stopwords.words('english')
    data['cleaned'] = cleaned_df.apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    df = data['cleaned'].copy()
    df = pd.DataFrame(df)
    
    df['token'] = ''
    j=0
    for i in cleaned_df:
        df.token[j]=i.split()
        j = j+1
        
    f = cleaned_df[0].split()
    
    df['skills'] = ''
    j = 0
    for i in df.token:
        df['skills'][j] = extract_skills(df.token[j])
        j = j+1
    
    df['post'] = ''
    j = 0
    for i in df.token:
        df['post'][j] = extract_post(df.token[j])
        j = j+1
    df['name'] = ''
    j = 0
    for i in df.token:
        df['name'][j] = contactDetails(df.token[j])
        j = j+1    
    
    resume = df.drop(columns={'cleaned','token'}, axis = 1)
    
    resume['lem'] = ''
    j = 0
    for i in resume.post:
        resume['lem'][j] = lem(resume.post[j])
        j = j+1
        
    
    
    resume['stem'] = ''
    j = 0
    for i in resume.lem:
        resume['stem'][j] = stem(resume.lem[j])
        j = j+1
    
    
    resume['designation'] =''
    j = 0
    for i in resume.post:
        resume['designation'][j] = extract_designation(resume.stem[j])
        j = j+1
    
    data = resume[['name','skills','designation']]
    
    d = data.designation.copy()
    
    
    
    

    var_mod = ['designation']
    le = LabelEncoder()
    for i in var_mod:
        data[i] = le.fit_transform(data[i])
    
    
    data = pd.concat([data,d],axis=1,ignore_index=False)
    
    return data

In [6]:
x = r'E:\DATA SETS\Resumes'
resume_classifier

<function __main__.resume_classifier(source_directory)>

In [7]:
import pickle

In [8]:
pickle.dump(resume_classifier(x), open('resume.pkl','wb'))

In [18]:
model=pickle.load(open('resume.pkl','rb'))

In [ ]:
model